# VGN Evaluation

In [ ]:
import os
os.chdir('..')

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rospy

from vgn import benchmark

In [ ]:
class SimpleNamespace(object):
    def __init__ (self, **kwargs):
        self.__dict__.update(kwargs)

## Network

Compute average loss and accuracy on test dataset.

## Benchmark

Run a clutter removal experiment and compute the following metrics:

- Success rate: fraction of successful grasp attempts
- Percent cleared: fraction of objects that were removed
- Planning time: average grasp planning time

In [ ]:
rospy.init_node("benchmark", anonymous=True)

In [ ]:
model = Path("data/runs/200602-0959,dataset=blocks,augment=True,net=conv,batch_size=32,lr=3e-04,balanced/vgn_conv_30.pth")
logdir = Path("data/experiments/mehmeh")
object_set = "blocks"

In [ ]:
args = SimpleNamespace(
    model=model,
    logdir=logdir,
    rounds=5,
    object_set=object_set,
    object_count=5,
    seed=1,
    sim_gui=True,
    rviz=True,
)
benchmark.main(args)

In [ ]:
success_rate, percent_cleared, planning_time = benchmark.metrics(log)

print("Success rate   ", success_rate)
print("Percent cleared", percent_cleared)
print("Planning time  ", planning_time)

## Learning Curves

Plot network loss and grasp metrics vs training epochs.

In [ ]:
exp_name = "05_blocks_balanced_augmented_noborder"
run_dir = Path("data/runs/200602-0959,dataset=blocks,augment=True,net=conv,batch_size=32,lr=3e-04,balanced")
object_set = "blocks"
epochs_to_evaluate = np.arange(1,30)

In [ ]:
for epoch in epochs_to_evaluate:
    model = run_dir / ("vgn_conv_" + str(epoch) + ".pth")
    logdir = Path("data") / "experiments" / exp_name / object_set / str(epoch)    
    
    if logdir.exists():
        continue  # manually delete folder to rerun benchmark
    
    args = SimpleNamespace(
        model=model,
        logdir=logdir,
        rounds=40,
        object_set=object_set,
        object_count=5,
        seed=1,
        sim_gui=False,
        rviz=False,
    )
    benchmark.main(args)

In [ ]:
df = pd.read_csv(run_dir / "train" / "loss.csv")
train_epochs = df["Step"].to_numpy()
train_loss = df["Value"].to_numpy()

df = pd.read_csv(run_dir / "validation" / "loss.csv")
val_epochs = df["Step"].to_numpy()
val_loss = df["Value"].to_numpy()

In [ ]:
root = Path("data") / "experiments" / exp_name / object_set
success_rates = []
for epoch in epochs_to_evaluate:
    log_dir = root / str(epoch)
    success_rate, _, _ = benchmark.metrics(log_dir)
    success_rates.append(success_rate)

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

l1, = ax1.plot(train_epochs, train_loss, color="C0")
l2,= ax1.plot(val_epochs, val_loss, color="C1")
l3, = ax2.plot(epochs_to_evaluate, success_rates, color="C2")

ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax2.set_ylabel("%")

ax1.legend([l1, l2], ["train", "validation"], loc="lower left")
ax2.legend([l3], ["success rate"], loc="upper right")

fig.suptitle(exp_name)
fig_path = Path.home() / "Desktop" / (exp_name + ".png")
plt.savefig(str(fig_path))
plt.show()